In [1]:
# https://wikidocs.net/60572

---
# 03. 소프트맥스 회귀의 비용 함수 구현하기

In [1]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

In [2]:
# 소프트맥수 함수의 비용 함수를 로우-레벨로 구현해보자

z = torch.FloatTensor([1,2,3])

In [3]:
# 위에서 정의한 텐서를 입력으로하는 소프트맥스 함수를 만들자

hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
# 원소들의 합을 확인하자
print(hypothesis.sum())

tensor(1.)


In [8]:
# ------------------
# 비용 함수를 직접 구현하자
# 3*5 크기의 텐서 만들기
z = torch.rand(3, 5, requires_grad=True)
print(z)

tensor([[0.4550, 0.5725, 0.4980, 0.9371, 0.6556],
        [0.3138, 0.1980, 0.4162, 0.2843, 0.3398],
        [0.5239, 0.7981, 0.7718, 0.0112, 0.8100]], requires_grad=True)


In [20]:
# 이 텐서를 입력으로 하는 소프트맥수 함수를 적용
# 단 각 샘플에 대해 소프트맥스를 적용하여야 하므로 dim = 1 으로 적용
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
print([i.sum() for i in hypothesis])

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward>)
[tensor(1., grad_fn=<SumBackward0>), tensor(1., grad_fn=<SumBackward0>), tensor(1., grad_fn=<SumBackward0>)]


In [30]:
# 각 샘플에 대하여 임의의 레이블을 만들자
y = torch.randint(5, (3,)).long()   # long 타입으로 변환
print(y)

tensor([3, 0, 4])


In [39]:
# 각 레이블에 대하여 원핫 인코딩을 수행
y_one_hot = torch.zeros_like(hypothesis)    # 닮고 싶은 크기의 array 의 zero array를 만든다.
print(y_one_hot.scatter_(1, y.unsqueeze(1), 1))
# scatter_ 은 inplace=True인 상태로 scatter와 다르게 바로 해당 tensor에 적용된다.

print(y.unsqueeze(1))

tensor([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])
tensor([[3],
        [0],
        [4]])


In [41]:
# 비용 함수 코드로 구현
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4478, grad_fn=<MeanBackward0>)


---
## 하이레벨로 소프트맥스 비용 함수 구현하기

In [44]:
# F.softmax() + torch.log() = F.log_softmax()
# ---------------------------------------------------
# low level 에서는 소프트맥스 함수의 결과에 로그를 씌울 때 다음과 같이 사용했다
torch.log(F.softmax(z, dim=1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward>)

In [45]:
# 그런데 토치에서는 두 개의 함수를 결합한 F.log_softmax 라는 도구를 제공한다
# High level
F.log_softmax(z, dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward>)

In [ ]:
# 위 두 출력 결과가 동일한 것을 확인

In [46]:
# F.log_softmax() + F.nll_loss() = F.cross_entropy()
# ---------------------------------------------------
# 이번에는 비용함수를 보자

# low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4478, grad_fn=<MeanBackward0>)

In [48]:
# 위 수식에서 torch.log(F.softmax(z, dim=1))를 방금 배운 F.log_softmax()로 대체할 수 있다.

(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4478, grad_fn=<MeanBackward0>)

In [ ]:
# 두 출력 결과가 동일한 것을 확인

In [49]:
# --------------------------------------------------
# F.nll_loss() 사용하기

# 더 간단하게 Negative log Likelihood를 사용할 수 있다. --> nll_loss
# nll_loss는 원-핫 벡터를 넣을 필요 없이 바로 실제값을 인자로 사용한다
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4478, grad_fn=<NllLossBackward>)

In [51]:
# --------------------------------------------------
# F.cross_entropy() 사용하기

# F.cross_entropy는 log_softmax와 nll_loss를 포함하고 있다
F.cross_entropy(z, y)

tensor(1.4478, grad_fn=<NllLossBackward>)